In [1]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import END, START
from langgraph.graph.state import StateGraph
from langgraph.graph.message import add_messages, AnyMessage
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import BaseMessage
import os
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["LANGSMITH_PROJECT"] = "TestProject"

In [3]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"

In [4]:
llm = ChatGroq(model = "llama-3.3-70b-versatile")

In [5]:
class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [6]:
@tool
def add(a: float, b: float):
    """Add two numbers"""
    return a + b

In [7]:
tools = [add]

In [8]:
llm_with_tool = llm.bind_functions([add])

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_31016\132165304.py:1: LangChainDeprecationWarning: The method `ChatGroq.bind_functions` was deprecated in langchain-groq 0.2.1 and will be removed in 1.0.0. Use :meth:`~langchain_groq.chat_models.ChatGroq.bind_tools` instead.
  llm_with_tool = llm.bind_functions([add])


In [9]:
def call_llm_model(state: State):
    return {"messages": [llm_with_tool.invoke(state["messages"])]}

In [10]:
builder = StateGraph(State)

builder.add_node("call_llm_model", call_llm_model)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "call_llm_model")
builder.add_conditional_edges("call_llm_model", tools_condition)
builder.add_edge("tools", "call_llm_model")

graph = builder.compile()

In [11]:
message = graph.invoke({"messages": "what is machine learning?"})
print(message)

{'messages': [HumanMessage(content='what is machine learning?', additional_kwargs={}, response_metadata={}, id='9f9a4525-2601-437b-8e1b-2eeee9d3eb87'), AIMessage(content='Machine learning is a subset of artificial intelligence (AI) that involves the use of algorithms and statistical models to enable machines to perform tasks without being explicitly programmed. It allows systems to learn from data, identify patterns, and make decisions or predictions based on that data. Machine learning involves training a model on a dataset, which enables the model to make predictions or take actions based on new, unseen data. The goal of machine learning is to develop models that can improve their performance on a task over time, without being explicitly programmed for that task.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 108, 'prompt_tokens': 217, 'total_tokens': 325, 'completion_time': 0.287588282, 'prompt_time': 0.018374173, 'queue_time': 0.085972648, 'total_ti

In [12]:
response = graph.invoke({"messages": "what is 2 plus 2?"})
print(response)

{'messages': [HumanMessage(content='what is 2 plus 2?', additional_kwargs={}, response_metadata={}, id='04681daa-6eb0-44ed-9400-705583d88d82'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'sttd8jzdw', 'function': {'arguments': '{"a":2,"b":2}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 220, 'total_tokens': 238, 'completion_time': 0.065507209, 'prompt_time': 0.012081296, 'queue_time': 0.092066274, 'total_time': 0.077588505}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_9e1e8f8435', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--dc63963a-7017-41da-9b1c-41723673ad46-0', tool_calls=[{'name': 'add', 'args': {'a': 2, 'b': 2}, 'id': 'sttd8jzdw', 'type': 'tool_call'}], usage_metadata={'input_tokens': 220, 'output_tokens': 18, 'total_tokens': 238}), ToolMessage(content='4.0', name='add', id='338edcb1-cf8e-4084-8b77-908195fa2fe9', tool_call_id